<a href="https://colab.research.google.com/github/HeeSang1996/CapstoneProjectCocktail/blob/master/ML_Assignment9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files

uploaded = files.upload()

Saving mnist.csv to mnist.csv


In [0]:
import matplotlib.pyplot as plt
import numpy as np

file_data   = "mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images

#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):
    data_normalized = (data - min(data)) / (max(data) - min(data))
    return(data_normalized)

#
# example of distance function between two vectors x and y
#
def distance(x, y):
    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)
    return(s)

#
# make a matrix each column of which represents an images in a vector form
#
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:

    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)

    list_label[count]       = label
    list_image[:, count]    = im_vector

    count += 1

In [0]:
#num of each layer
num_input_layer = size_row*size_col
num_first_hidden_layer = 196
num_second_hidden_layer = 49
num_output_layer = 10

#bias
x_bias = 1.0
y_bias = 1.0
z_bias = 1.0

In [0]:
#make random theta with normal distribution with bias
initial_theta_u = np.random.normal(0.0,1.0,((num_input_layer+1),num_first_hidden_layer))
initial_theta_v = np.random.normal(0.0,1.0,((num_first_hidden_layer+1),num_second_hidden_layer))
initial_theta_w = np.random.normal(0.0,1.0,((num_second_hidden_layer+1),num_output_layer))

In [0]:
theta_u = initial_theta_u
theta_v = initial_theta_v
theta_w = initial_theta_w

In [0]:
def sigmoidFunc(data):
    sigmoidVal = 1/(1+np.exp(-data))
    return(sigmoidVal)

In [0]:
def neuralNetwork(start,end):
    list_image_with_bias  = np.empty((num_input_layer+1, end-start), dtype=float)
    list_y = np.empty((num_first_hidden_layer,end-start), dtype=float)
    list_y_with_bias = np.empty((num_first_hidden_layer+1, end-start), dtype=float)
    list_z = np.empty((num_second_hidden_layer, end-start), dtype=float)
    list_z_with_bias = np.empty((num_second_hidden_layer+1, end-start), dtype=float)
    list_h = np.empty((num_output_layer, end-start), dtype=float)
    list_label_ = np.empty((num_output_layer, end-start), dtype=float)

    count = 0
    for i in range(start,end):
        #add bias for x
        list_image_with_bias[:, count] = np.insert(list_image[:,i],0,x_bias)
        #u: x -> y
        list_y[:,count]=np.dot(list_image_with_bias[:,count],theta_u)
        list_y[:,count]=sigmoidFunc(list_y[:,count])
        #add bias for y
        list_y_with_bias[:,count]=np.insert(list_y[:,count],0,y_bias)
        #v: y -> z
        list_z[:,count]=np.dot(list_y_with_bias[:,count],theta_v)
        list_z[:,count]=sigmoidFunc(list_z[:,count])
        #add bias for z
        list_z_with_bias[:,count]=np.insert(list_z[:,count],0,z_bias)
        #w: z -> h
        list_h[:,count]=np.dot(list_z_with_bias[:,count],theta_w)
        list_h[:,count]=sigmoidFunc(list_h[:,count])
        #label list
        label=np.zeros(10, dtype=float)
        label[list_label[i]]=1.0
        list_label_[:,count]=label

        count = count +1

    return(list_image_with_bias,list_y_with_bias,list_z_with_bias,list_h,list_label_)

In [0]:
def lossFunction(output_list,start,end):
    loss = 0.0
    count = 0
    accuracy_count=0
    for i in range(start,end):
        label=np.zeros(10, dtype=float)
        label[list_label[i]]=1.0
        if list_label[i]==np.argmax(output_list[:,count]):
            accuracy_count = accuracy_count + 1;
        loss += np.sum((-label)*np.log(output_list[:,count])-(1-label)*np.log(1-output_list[:,count]))
        count = count+1
    loss = loss/(end - start)
    accuracy = accuracy_count/(end - start)
    return(loss,accuracy)

In [0]:
#step size and step range
step_size = 0.59
step_range = 100

training_size = 6000
testing_size = 4000

In [0]:
training_loss_value_list = []
testing_loss_value_list = []
training_accuracy_value_list = []
testing_accuracy_value_list = []

for i in range(1):
    (x_list_training,y_list_training,z_list_training,training_output_list,training_label_list) \
    = neuralNetwork(0,training_size)
    (training_loss_value,training_accuracy_value)=lossFunction(training_output_list,0,training_size)
    training_loss_value_list.append(training_loss_value)
    training_accuracy_value_list.append(training_accuracy_value)

    (_,_,_,testing_output_list,_) = neuralNetwork(training_size,num_image)
    (testing_loss_value,testing_accuracy_value)=lossFunction(testing_output_list,training_size,num_image)
    testing_loss_value_list.append(testing_loss_value)
    testing_accuracy_value_list.append(testing_accuracy_value)

    #theta_u = theta_u - step_size*
    theta_v = theta_v - np.transpose(step_size*np.dot(np.dot(theta_w[1:,:],(training_output_list-training_label_list))\
    *(z_list_training[1:,:]*(1-z_list_training[1:,:])),np.transpose(y_list_training)))
    theta_w = theta_w - np.transpose(step_size*\
                                     np.dot((training_output_list-training_label_list),\
                                            np.transpose(z_list_training))/training_size)
#for_test_z = np.dot(z_list_training.sum(axis = 1).reshape(50,1),training_output_list.sum(axis=1).reshape(1,10))

#1. Plot the loss curve

#2. Plot the accuracy curve

#3. Plot the accuracy value

#4. Plot the classification example